# 02/19/2022 SSL Interception

In [1]:
import pandas as pd
import os

In [12]:
df = pd.read_parquet("/home/ubuntu/GitLab/ssl_interception/historical_dfs/anon.ssl_20220219_0800-0830-0500.log_pos_intercept.parquet")

In [13]:
df

,index,domain,issuer_O,count,historical_ca
0,1,webproxy.allina.com,EMR Direct,2,DigiCert Inc
1,3,epic-ce-prod.tvc.org,EMR Direct,1,Sectigo Limited
2,15,epiccesoap.altru.org,EMR Direct,2,"GoDaddy.com, Inc."
3,18,ceprd.sjhsyr.org,EMR Direct,2,None
4,23,yrmcsfd.yumaregional.org,EMR Direct,2,None
...,...,...,...,...,...
116,451,careeverywhere.childrensdayton.org,MaxMD,2,None
117,454,sfd.fairview.org,EMR Direct,1,GlobalSign nv-sa
118,455,devops.shottracker.com,N=3x1M07 5EbvZ8tQOb 1V 4S1O61pWj 1w afGABq.48F...,1,DigiCert Inc
119,456,careeverywhere.phs.org,EMR Direct,2,DigiCert Inc


In [14]:
df = df[df["issuer_O"] != "EMR Direct"]

In [20]:
df.reset_index(inplace=True, drop=True)
df.drop("index", axis=1, inplace=True)

/home/ubuntu/local_pip/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
df

,domain,issuer_O,count,historical_ca
0,gld.push.samsungosp.com,SAMSUNG ELECTRONICS CO.,1,Sectigo Limited
1,careeverywhereprod.lhs.org,MaxMD,1,None
2,5aOZBG92.vt.edu,SomeOrganization,11,None
3,usacmg.cloudapp.net,"N=2K3sL5a 69b 14i 3s,DC=skanska,DC=or",15,None
4,6Q2kLQfg.virginia.edu,SomeOrganization,535,None
...,...,...,...,...
57,www.puofv55mnpzshw.com,N=www.tdb6lel3ehsdiabn.co,1,Unable to find
58,hh.prod.sonyentertainmentnetwork.com,Sony Computer Entertainment Inc.,8,None
59,grs.dbankcloud.cn,Huawei,10,None
60,careeverywhere.childrensdayton.org,MaxMD,2,None


In [30]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find")]
df_not_none

,domain,issuer_O,count,historical_ca
0,gld.push.samsungosp.com,SAMSUNG ELECTRONICS CO.,1,Sectigo Limited
10,grs.dbankcloud.com,Huawei,54,GlobalSign nv-sa
16,ce.alegent.com,MaxMD,1,COMODO CA Limited
30,careeverywhere.gmh.edu,MaxMD,2,"Entrust, Inc."
41,passport.migu.cn,ZYHZYFZX,4,DigiCert Inc
61,devops.shottracker.com,N=3x1M07 5EbvZ8tQOb 1V 4S1O61pWj 1w afGABq.48F...,1,DigiCert Inc


In [2]:
path = "/home/ubuntu/GitLab/ssl_interception/historical_dfs/anon.ssl_20220219_0000-0030-0500.log_pos_intercept.parquet"
df = pd.read_parquet(path)

In [5]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find")]
df_not_equal = df_not_none[df_not_none["issuer_O"] != df_not_none["historical_ca"]]
df_not_equal

,index,domain,issuer_O,count,historical_ca
23,58,.wikisource.org,GlobalSign nv-sa,4,Let's Encrypt
54,114,.wiktionary.org,GlobalSign nv-sa,17,Let's Encrypt
189,433,.emiratesnbd.com,COMODO CA Limited,1,DigiCert Inc
219,503,.wikiquote.org,GlobalSign nv-sa,5,Let's Encrypt


# Combining DFs

In [1]:
import os 
import pandas as pd

In [5]:
data_path = "/home/ubuntu/GitLab/ssl_interception/intercept_dfs/march_6_intercept_df/"
df = pd.read_parquet(data_path + os.listdir(data_path)[0])


In [4]:
def get_intercept_df(data_path: str, base_df: pd.DataFrame):
    files = os.listdir(data_path)
    intercept_df = base_df
    for file in files[1:]:
        tmp_df = pd.read_parquet(data_path + file)
        intercept_df = pd.concat([base_df, tmp_df]).drop_duplicates()
        intercept_df.reset_index(inplace=True, drop=True)
    return intercept_df

In [7]:
df = get_intercept_df(data_path, df)
df

,index,domain,issuer_O,count,historical_ca
0,3,perfva.cv.nrao.edu,SomeOrganization,45,"{'Sectigo Limited', 'COMODO CA Limited', '""GoD..."
1,29,5aOZBG92.vt.edu,SomeOrganization,30,Amazon
2,45,usacmg.cloudapp.net,"N=2K3sL5a 69b 14i 3s,DC=skanska,DC=or",13,Unable to find
3,49,6Q2kLQfg.virginia.edu,SomeOrganization,116,Unable to find
4,53,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,3,Let's Encrypt
...,...,...,...,...,...
70,336,www.7q6y.com,N=www.gdfyejzac.co,1,None
71,370,grs.dbankcloud.cn,Huawei,13,None
72,372,US.info.lgsmartad.com,LG Electronics Inc.,2,None
73,376,us.ad.lgsmartad.com,LG Electronics Inc.,2,None


In [37]:
df_aruba = df[df["issuer_O"] == "Aruba Networks"]
df_aruba

,index,domain,issuer_O,count,historical_ca
95,130,3X0iXA2I.virginia.edu,Aruba Networks,2,Unable to find
97,146,1cBIfGed.virginia.edu,Aruba Networks,2,Unable to find
136,507,6Jf8LZMH.virginia.edu,Aruba Networks,2,Unable to find


In [50]:
df.groupby("issuer_O")["domain"].agg("count").reset_index(name="count").sort_values(by="count", ascending=False)

,issuer_O,count
33,Sony Computer Entertainment Inc.,34
0,Apple Inc.,23
31,Samsung Electronics,16
32,SomeOrganization,12
17,N=4K2VsoWa,4
1,Aruba Networks,3
5,DigiCert Inc,3
9,Huawei,3
12,LG Electronics Inc.,3
29,Qterics Inc.,2


In [54]:
df_qterics = df[df["issuer_O"] == "Qterics Inc."]
df_qterics

,index,domain,issuer_O,count,historical_ca
108,255,00199D0302-da.support.supportview.com,Qterics Inc.,1,Unable to find
118,327,00199D0302-drmp.support.supportview.com,Qterics Inc.,1,Unable to find


In [46]:
df_open = df[df["issuer_O"] == "Open Whisper Systems"]
df_open

,index,domain,issuer_O,count,historical_ca
65,404,api.directory.signal.org,Open Whisper Systems,1,Unable to find
129,412,api.directory.signal.org,Open Whisper Systems,2,Unable to find


In [26]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find")]

In [29]:
df_not_none

,index,domain,issuer_O,count,historical_ca


In [56]:
df_brocade = df[df["issuer_O"] == "Brocade Communications Systems"]
df_brocade

,index,domain,issuer_O,count,historical_ca
110,282,.31.81.50.9.,Brocade Communications Systems,1,Unable to find


In [8]:
df = get_intercept_df(data_path, df)
df

,index,domain,issuer_O,count,historical_ca
0,3,perfva.cv.nrao.edu,SomeOrganization,45,None
1,27,.128.143.5.253.,SomeOrganization,9,Unable to find
2,31,5aOZBG92.vt.edu,SomeOrganization,30,None
3,47,usacmg.cloudapp.net,"N=2K3sL5a 69b 14i 3s,DC=skanska,DC=or",13,None
4,51,6Q2kLQfg.virginia.edu,SomeOrganization,116,None
...,...,...,...,...,...
70,336,www.7q6y.com,N=www.gdfyejzac.co,1,None
71,370,grs.dbankcloud.cn,Huawei,13,None
72,372,US.info.lgsmartad.com,LG Electronics Inc.,2,None
73,376,us.ad.lgsmartad.com,LG Electronics Inc.,2,None


In [10]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find")]
df_not_none

,index,domain,issuer_O,count,historical_ca
0,3,perfva.cv.nrao.edu,SomeOrganization,45,"{'Sectigo Limited', 'COMODO CA Limited', '""GoD..."
1,29,5aOZBG92.vt.edu,SomeOrganization,30,Amazon
4,53,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,3,Let's Encrypt
6,69,www.lqzes32o77b6yy.com,N=www.kkncx5u5kz5jwhwsjff.co,1,None
7,75,www.5amnkfzpclosajopkqrugv.com,N=www.uvxeuit3ikq.co,1,None
8,79,stasas.sascms.net,CommScope,124,None
9,84,www.fwhfdg2.com,N=www.zlc7ieu6gk4f.co,1,None
10,85,collector.appoptics.com,Librato Inc.,1,Amazon
11,86,31BCnWjt.,STARS GROUP,1,None
12,91,prov.prod.granite.clouddevice.io,Honeywell International Inc.,1,None


# March 2022 Data

In [15]:
print(df.iloc[26])
print("Historical CAs: " + df.iloc[26]["historical_ca"])

index                                                          267
domain                                            rprm.cv.nrao.edu
issuer_O                                                   Polycom
count                                                           28
historical_ca    {'Sectigo Limited', 'COMODO CA Limited', '"GoD...
Name: 26, dtype: object
Historical CAs: {'Sectigo Limited', 'COMODO CA Limited', '"GoDaddy.com, Inc."'}


# 2/25/2022

In [4]:
data_path = "/home/ubuntu/GitLab/ssl_interception/intercept_dfs/feb_25_intercept_df/"
df = pd.read_parquet(data_path + os.listdir(data_path)[0])

In [14]:
df = get_intercept_df(data_path, df)
df.shape

(44, 5)

In [31]:
df_not_none = df[~df["historical_ca"].isnull()]
df_not_none

,index,domain,issuer_O,count,historical_ca
0,41,5aOZBG92.vt.edu,SomeOrganization,130,Amazon
2,64,6Q2kLQfg.virginia.edu,SomeOrganization,71,"{'Internet2', 'ChainedSSL', 'DigiCert Inc'}"
3,67,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,1,Let's Encrypt
4,78,wikisource.org,GlobalSign nv-sa,2,"{""Let's Encrypt"", 'GlobalSign nv-sa', '""GeoTru..."
6,105,collector.appoptics.com,Librato Inc.,1,Amazon
7,129,53ih8wpq.virginia.edu,N=websync.cavaliers.DVSport.co,3,"{'Internet2', 'ChainedSSL', 'DigiCert Inc'}"
8,137,US.lgtvsdp.com,LG Electronics Inc.,4,"{'""thawte, Inc.""', '""Thawte, Inc.""', 'Amazon',..."
9,154,6mUpT5Jp.virginia.edu,SomeOrganization,1,"{'Internet2', 'ChainedSSL', 'DigiCert Inc'}"
10,155,00199D0306-drmp.support.supportview.com,Qterics Inc.,1,"{'IdenTrust', 'HydrantID (Avalanche Cloud Corp..."
12,165,expressway.vatelemed.net,Temporary CA a0305fb9-1d21-4142-9f97-e1c6efdab8f,9,"{'Internet2', 'Sectigo Limited'}"


df.iloc[42]["historical_ca"]

# 3/20/2022

In [5]:
data_path = "/home/ubuntu/GitLab/ssl_interception/intercept_dfs/march_20_intercept_df/"
df = pd.read_parquet(data_path + os.listdir(data_path)[0])

In [6]:
df = get_intercept_df(data_path, df)
df.head(60)

,index,domain,issuer_O,count,historical_ca
0,30,5aOZBG92.vt.edu,SomeOrganization,33,Amazon
1,47,1ghbunIw.virginia.edu,Internet2,1,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
2,58,6Q2kLQfg.virginia.edu,SomeOrganization,17,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
3,61,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,2,Let's Encrypt
4,76,128.143.249.67.,SomeOrganization,5,Unable to find
5,77,5TA7alcA.virginia.edu,SomeOrganization,2,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
6,83,stasas.sascms.net,CommScope,133,Unable to find
7,92,lfodown01-b.cloudsink.net,CrowdStrike Inc.,1,"{'GlobalSign nv-sa', 'DigiCert Inc', '""CrowdSt..."
8,94,collector.appoptics.com,Librato Inc.,1,Amazon
9,112,ts01-b.cloudsink.net,CrowdStrike Inc.,1,"{'GlobalSign nv-sa', 'DigiCert Inc', '""CrowdSt..."


In [7]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find") & (df["issuer_O"] != "SomeOrganization")]
df_not_none

,index,domain,issuer_O,count,historical_ca
1,47,1ghbunIw.virginia.edu,Internet2,1,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
3,61,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,2,Let's Encrypt
7,92,lfodown01-b.cloudsink.net,CrowdStrike Inc.,1,"{'GlobalSign nv-sa', 'DigiCert Inc', '""CrowdSt..."
8,94,collector.appoptics.com,Librato Inc.,1,Amazon
9,112,ts01-b.cloudsink.net,CrowdStrike Inc.,1,"{'GlobalSign nv-sa', 'DigiCert Inc', '""CrowdSt..."
10,119,US.lgtvsdp.com,LG Electronics Inc.,2,"{'DigiCert Inc', '""thawte, Inc.""', 'Amazon', '..."
13,144,expressway.vatelemed.net,Temporary CA a0305fb9-1d21-4142-9f97-e1c6efdab8f,11,"{'Internet2', 'Sectigo Limited'}"
14,194,6IfzS4h4.virginia.edu,Brocade Communications Systems,1,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
15,207,discovery.syncthing.net,N=4K2VsoWa,16,COMODO CA Limited
18,256,grs.platform.dbankcloud.ru,Huawei,1,GlobalSign nv-sa


In [15]:
print(df.iloc[77])
print(df.iloc[77]["historical_ca"])

index                                                          446
domain                                                  paypal.com
issuer_O                                      Symantec Corporation
count                                                            3
historical_ca    {'The USERTRUST Network', '"VeriSign, Inc."', ...
Name: 77, dtype: object
{'The USERTRUST Network', '"VeriSign, Inc."', 'Symantec Corporation', '"cPanel, Inc."', 'VeriSign Trust Network', 'Akamai Technologies Inc', 'general@ipsca.com C.I.F.  B-B62210695', 'Cybertrust Inc', 'DigiCert Inc', "Let's Encrypt"}


# 3/31/2022

In [14]:
import pandas as pd
import os
data_path = "/home/ubuntu/GitLab/ssl_interception/intercept_dfs/march_31_intercept_df/"
df = pd.read_parquet(data_path + os.listdir(data_path)[0])

In [15]:
def get_intercept_df(data_path: str, base_df: pd.DataFrame):
    files = os.listdir(data_path)
    intercept_df = base_df
    for file in files[1:]:
        tmp_df = pd.read_parquet(data_path + file)
        intercept_df = pd.concat([base_df, tmp_df]).drop_duplicates()
        intercept_df.reset_index(inplace=True, drop=True)
    return intercept_df

In [16]:
df = get_intercept_df(data_path, df)
df

,index,domain,issuer_O,count,historical_ca
0,0,1.441512ffb9808d226c101255fe50159456e28d2a8c1b...,N=FRCGATN93004.dir.svc.accenture.com Root,1,Unable to find
1,3,www.7grkkdp6k2clfjrixzzzwxw.com,N=www.ldqxhcng.co,1,Unable to find
2,13,128.143.204.242.,SomeOrganization,1,Unable to find
3,25,5aOZBG92.vt.edu,SomeOrganization,122,Amazon
4,40,6Q2kLQfg.virginia.edu,SomeOrganization,17,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
...,...,...,...,...,...
128,578,grs.dbankcloud.cn,Huawei,12,"{'GlobalSign nv-sa', 'China Financial Certific..."
129,581,US.info.lgsmartad.com,LG Electronics Inc.,2,"{'DigiCert Inc', '""Thawte, Inc.""', '""thawte, I..."
130,587,bmecmg.cloudapp.net,"N=283Lr0 4zOfx 6Nwi 1V,DC=BartonMalow,DC=co",30,Unable to find
131,589,us.ad.lgsmartad.com,LG Electronics Inc.,1,"{'DigiCert Inc', '""Thawte, Inc.""', '""thawte, I..."


In [17]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find") & (df["issuer_O"] != "SomeOrganization")]
df_not_none

,index,domain,issuer_O,count,historical_ca
5,44,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,1,Let's Encrypt
9,71,collector.appoptics.com,Librato Inc.,3,Amazon
10,102,federation.eams.army.mil,U.S. Government,16,"{'IdenTrust', 'U.S. Government'}"
12,119,expressway.vatelemed.net,Temporary CA a0305fb9-1d21-4142-9f97-e1c6efdab8f,9,"{'Sectigo Limited', 'Internet2'}"
14,146,gtpa1.izatcloud.net,QUALCOMM,1,"{'Symantec Corporation', 'COMODO CA Limited', ..."
...,...,...,...,...,...
126,557,adservice.google.com.x.7a0e41b40a7d904f0e0a937...,Cisc,1,"{'Equifax', 'Equifax Secure Inc.', 'DigiCert I..."
128,578,grs.dbankcloud.cn,Huawei,12,"{'GlobalSign nv-sa', 'China Financial Certific..."
129,581,US.info.lgsmartad.com,LG Electronics Inc.,2,"{'DigiCert Inc', '""Thawte, Inc.""', '""thawte, I..."
131,589,us.ad.lgsmartad.com,LG Electronics Inc.,1,"{'DigiCert Inc', '""Thawte, Inc.""', '""thawte, I..."


In [19]:
df_not_none.reset_index(inplace=True, drop=True)

In [25]:
df_not_none.head(20)

,index,domain,issuer_O,count,historical_ca
0,44,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,1,Let's Encrypt
1,71,collector.appoptics.com,Librato Inc.,3,Amazon
2,102,federation.eams.army.mil,U.S. Government,16,"{'IdenTrust', 'U.S. Government'}"
3,119,expressway.vatelemed.net,Temporary CA a0305fb9-1d21-4142-9f97-e1c6efdab8f,9,"{'Sectigo Limited', 'Internet2'}"
4,146,gtpa1.izatcloud.net,QUALCOMM,1,"{'Symantec Corporation', 'COMODO CA Limited', ..."
5,154,ecs.office.com,Cisc,1,"{'DigiCert Inc', 'Microsoft Corporation'}"
6,172,teams.microsoft.com,Cisc,23,"{'DigiCert Inc', 'GlobalSign nv-sa', 'Microsof..."
7,187,cloudfront.net,Symantec Corporation,5,"{'Symantec Corporation', 'COMODO CA Limited', ..."
8,188,4UsZ5rK6.virginia.edu,Aruba Networks,2,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
9,194,discovery.syncthing.net,N=4K2VsoWa,15,COMODO CA Limited


In [28]:
df_not_none.iloc[21:]

,index,domain,issuer_O,count,historical_ca
21,18,config.teams.microsoft.com,Cisc,3,"{'DigiCert Inc', 'Microsoft Corporation'}"
22,25,storage.live.com,Cisc,1,Microsoft Corporation
23,35,wmsauthdev.wmsweb.afncr.af.mil,U.S. Government,2,U.S. Government
24,52,gtp1.izatcloud.net,QUALCOMM,1,"{'Symantec Corporation', 'COMODO CA Limited', ..."
25,64,px.ads.linkedin.com,Cisc,4,DigiCert Inc
26,72,simhost-0d0c9b920da73ce12.agni.secondlife.io,Linden Lab,9,"{'DigiCert Inc', 'Amazon'}"
27,74,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,2,Let's Encrypt
28,109,collector.appoptics.com,Librato Inc.,1,Amazon
29,111,www.facebook.com,Cisc,1,"{'Equifax Secure Inc.', 'VeriSign Trust Networ..."
30,127,ingress1.virgosvs.com,Virgo Surgical Video Systems,1,"{'COMODO CA Limited', 'Sectigo Limited'}"


In [30]:
df_not_none[df_not_none["issuer_O"] == "Cisc"]

,index,domain,issuer_O,count,historical_ca
5,154,ecs.office.com,Cisc,1,"{'DigiCert Inc', 'Microsoft Corporation'}"
6,172,teams.microsoft.com,Cisc,23,"{'DigiCert Inc', 'GlobalSign nv-sa', 'Microsof..."
12,259,presence.teams.microsoft.com,Cisc,1,Microsoft Corporation
13,274,autodiscover-s.outlook.com,Cisc,24,DigiCert Inc
15,289,df.trap.teams.microsoft.com,Cisc,1,Microsoft Corporation
16,290,outlook.office365.com,Cisc,118,"{'DigiCert Inc', 'GlobalSign nv-sa', 'Microsof..."
20,13,m.adnxs.com,Cisc,1,"{'""GoDaddy.com, Inc.""', '""GeoTrust, Inc.""', 'S..."
21,18,config.teams.microsoft.com,Cisc,3,"{'DigiCert Inc', 'Microsoft Corporation'}"
22,25,storage.live.com,Cisc,1,Microsoft Corporation
25,64,px.ads.linkedin.com,Cisc,4,DigiCert Inc


In [32]:
df_not_cisc = df_not_none[df_not_none["issuer_O"] != "Cisc"]
df_not_cisc

,index,domain,issuer_O,count,historical_ca
0,44,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,1,Let's Encrypt
1,71,collector.appoptics.com,Librato Inc.,3,Amazon
2,102,federation.eams.army.mil,U.S. Government,16,"{'IdenTrust', 'U.S. Government'}"
3,119,expressway.vatelemed.net,Temporary CA a0305fb9-1d21-4142-9f97-e1c6efdab8f,9,"{'Sectigo Limited', 'Internet2'}"
4,146,gtpa1.izatcloud.net,QUALCOMM,1,"{'Symantec Corporation', 'COMODO CA Limited', ..."
7,187,cloudfront.net,Symantec Corporation,5,"{'Symantec Corporation', 'COMODO CA Limited', ..."
8,188,4UsZ5rK6.virginia.edu,Aruba Networks,2,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"
9,194,discovery.syncthing.net,N=4K2VsoWa,15,COMODO CA Limited
10,236,ingress2.virgosvs.com,Virgo Surgical Video Systems,2,"{'COMODO CA Limited', 'Sectigo Limited'}"
11,252,avcloud02.bd.com,Becton,8,Corporation Service Company


In [35]:
df_not_cisc[df_not_cisc["domain"] == "4m1g2IRz.virginia.edu"]

,index,domain,issuer_O,count,historical_ca
51,381,4m1g2IRz.virginia.edu,VMware Installe,2,"{'DigiCert Inc', 'ChainedSSL', 'Internet2'}"


# 4/1/2022

In [1]:
import pandas as pd
import os
data_path = "/home/ubuntu/GitLab/ssl_interception/intercept_dfs/april_1_intercept_df/"
df = pd.read_parquet(data_path + os.listdir(data_path)[0])

In [2]:
def get_intercept_df(data_path: str, base_df: pd.DataFrame):
    files = os.listdir(data_path)
    intercept_df = base_df
    for file in files[1:]:
        tmp_df = pd.read_parquet(data_path + file)
        intercept_df = pd.concat([base_df, tmp_df]).drop_duplicates()
        intercept_df.reset_index(inplace=True, drop=True)
    return intercept_df

In [4]:
df = get_intercept_df(data_path, df)
df

,index,domain,issuer_O,count,historical_ca
0,12,www.glraiu.com,N=www.rgu2zt7zpkfyxj23hr3.co,1,None
1,14,config.teams.microsoft.com,Cisc,3,"{'DigiCert Inc', 'Microsoft Corporation'}"
2,23,storage.live.com,Cisc,6,Microsoft Corporation
3,46,5aOZBG92.vt.edu,SomeOrganization,19,Amazon
4,66,6Q2kLQfg.virginia.edu,SomeOrganization,437,Unable to find
...,...,...,...,...,...
116,545,perfva.nrao.edu,SomeOrganization,1,"{'Sectigo Limited', 'COMODO CA Limited', '""GoD..."
117,573,grs.dbankcloud.cn,Huawei,8,"{'GlobalSign nv-sa', 'China Financial Certific..."
118,575,US.info.lgsmartad.com,LG Electronics Inc.,3,"{'""thawte, Inc.""', '""Thawte, Inc.""', 'DigiCert..."
119,583,bmecmg.cloudapp.net,"N=283Lr0 4zOfx 6Nwi 1V,DC=BartonMalow,DC=co",28,Unable to find


In [6]:
df_not_none = df[(df["historical_ca"] != "None") & (df["historical_ca"] != "Unable to find") & (df["issuer_O"] != "SomeOrganization")]
df_not_none

,index,domain,issuer_O,count,historical_ca
1,14,config.teams.microsoft.com,Cisc,3,"{'DigiCert Inc', 'Microsoft Corporation'}"
2,23,storage.live.com,Cisc,6,Microsoft Corporation
5,70,ols.chaosgroup.com,N=7Fw6H 2cMYVTs,3,Let's Encrypt
6,80,mosen.mt.co.kr,Turing Crypto GmbH,3,"{'Sectigo Limited', 'Turing Crypto GmbH'}"
9,111,collector.appoptics.com,Librato Inc.,1,Amazon
...,...,...,...,...,...
112,453,discovery-v4.syncthing.net,N=4K2VsoWa,8,COMODO CA Limited
113,464,gosca.ea.com,Electronic Arts,3,DigiCert Inc
117,573,grs.dbankcloud.cn,Huawei,8,"{'GlobalSign nv-sa', 'China Financial Certific..."
118,575,US.info.lgsmartad.com,LG Electronics Inc.,3,"{'""thawte, Inc.""', '""Thawte, Inc.""', 'DigiCert..."


,index,domain,issuer_O,count,historical_ca
0,12,www.glraiu.com,N=www.rgu2zt7zpkfyxj23hr3.co,1,None
1,14,config.teams.microsoft.com,Cisc,3,"{'DigiCert Inc', 'Microsoft Corporation'}"
2,23,storage.live.com,Cisc,6,Microsoft Corporation
3,46,5aOZBG92.vt.edu,SomeOrganization,19,Amazon
4,66,6Q2kLQfg.virginia.edu,SomeOrganization,437,Unable to find
...,...,...,...,...,...
116,545,perfva.nrao.edu,SomeOrganization,1,"{'Sectigo Limited', 'COMODO CA Limited', '""GoD..."
117,573,grs.dbankcloud.cn,Huawei,8,"{'GlobalSign nv-sa', 'China Financial Certific..."
118,575,US.info.lgsmartad.com,LG Electronics Inc.,3,"{'""thawte, Inc.""', '""Thawte, Inc.""', 'DigiCert..."
119,583,bmecmg.cloudapp.net,"N=283Lr0 4zOfx 6Nwi 1V,DC=BartonMalow,DC=co",28,Unable to find
